In [ ]:
device = "mps"

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import tqdm as tqdm
from ml_zoo import MNISTDataModule, MNISTDataModuleConfig

In [ ]:
dm = MNISTDataModule(
    MNISTDataModuleConfig(
        "data",
        batch_size=128,
        num_workers=4,
        pin_memory=True,
        persistent_workers=True,
    )
)
dm.prepare_data()
dm.setup()

train_loader = dm.train_dataloader()
val_loader = dm.val_dataloader()